In [ ]:
#1导包
import tets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

#2 定义超参
BATCH_SIZE = 8 #每次处理的数据数量
DEVICE = torch.device("cpu")#模型是在CPU还是GPU训练
EPOCHS = 3   #训练的轮次

#3 构建pipeline，对图像处理进行旋转等处理
pipeline = transforms.Compose([
    transforms.ToTensor(),#将图片转换成tensor格式
    transforms.Normalize((0.1307),(0.3081,))#正则化，出现过拟合，降低模型复杂度
])

#4 下载、加载数据集
from torch.utils.data import DataLoader
#下载数据集
train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline)
test_set = datasets.MNIST("data", train=False,download=True, transform=pipeline)
#加载数据集
train_loder = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)#将图片顺序打乱
test_loder = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)#将图片顺序打乱

# 显示图片
# with open("data/MNIST/raw/train-images-idx3-ubyte", "rb") as f:
#     file = f.read()
# image1 = [int(str(item).encode('ascii'), 16) for item in file[16:16+784]]
# # print(image1)
# import cv2
# import numpy as np
# image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)
# print(image1_np.shape)
# cv2.imwrite("1.jpg", image1_np)

# 5构建网络模型
class Digit(nn.Module):
    #模型初始化
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, 5) #1:灰度图片的通道，10：输出通道，5：卷积核数量
        self.conv2 = nn.Conv2d(10, 20, 3)#10:输入通道，20：输出通道，3：卷积核数量
        self.fc1 = nn.Linear(20*10*10, 500) #20*20*10:输入通道，500：输出通道
        self.fc2 = nn.Linear(500, 10)#500:输入通道，10：输出通道

    #正向传播
    def forward(self, x):
        input_size = x.size(0) #batch_size
        x = self.conv1(x)#输入：batch__size*1*28*28,输出：batch_size*10*24*24（卷积计算：28-5+1）
        x = F.relu(x)#激活函数
        x = F.max_pool2d(x, 2, 2)#池化层（降采样）输入：batch_size*10*24*24 输出：batch_size*10*12*12

        x = self.conv2(x)#输入batch_size*10*12*12 输出：batch_size*20*10*10(12-3+1)
        x = F.relu(x)#激活函数

        x = x.view(input_size, -1) #将二维图像矩阵转化成一维向量   -1：自动计算维度20*10*10=2000

        x = self.fc1(x) #输入：batch_size*2000 输出：batch_size*500
        x = F.relu(x)#激活函数

        x = self.fc2(x)#输入：batch_size*500 输出：batch_size*10

        output = F.log_softmax(x, dim=1) #计算分类后，每个数字的概率值
        return output


#6 定义优化器
model = Digit().to(DEVICE)#选择设备
optimizer = optim.Adam(model.parameters())#选择优化方法

#7 定义训练方法
def train_model(model, devide, train_loader, optimizer, epoch):
    #模型训练
    model.train()
    for batch_index, data, target in tets.datas_0:
        print(train_loader)
        print(type(target[0]))
        print(data[0])

        print(batch_index)
        #将数据部署到模型
        data, target = data.to(devide), target.to(devide)
        #梯度初始化为0
        optimizer.zero_grad()
        #训练后的结果
        output = model(data)
        #计算损失
        loss = F.cross_entropy(output, target)
        # #找到概率值最大的下标
        # pred = output.max(1, keepdim=True)#pred = output.argmax(dim=1)
        #反向传播
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print('Train Epoch : {} \t Loss : {:.6f}'.format(epoch, loss.item()))

# 8测试方法
def test_model(model, devide, test_loader):
    #模型验证
    model.eval()
    #正确率
    correct =0.0
    #测试损失
    test_loss = 0.0
    #进行测试
    with torch.no_grad(): #不会计算梯度，也不会反向传播
        for data, target in test_loader:
            #部署到device上
            data, target = data.to(devide), target.to(devide)
            #测试数据
            output = model(data)
            #计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            #找到概率值最大的下标
            pred = output.argmax(dim=1)
            # pred = output.max(1, keepdim=True)[0/1] "0":概率值 "1"：图片索引
            #
            # ？？？model.softmax()
            #累计正确率
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loder.dataset)
        print("Test —— Average loss : {:.4f}, Accuracy : {:.3f}\n".format(
            test_loss, 100.0 * correct / len(test_loader.dataset)))

#9 调用 7/8方法
for epoch in range(1, EPOCHS + 1):
    train_model(model, DEVICE, train_loder, optimizer, epoch)
    test_model(model, DEVICE, test_loder)